# Problem 20: Document clustering

_Version 1.4_

Suppose we have several documents and we want to _cluster_ them, meaning we wish to divide them into groups based on how "similar" the documents are. One question is what does it mean for two documents to be similar?

In this problem, you will implement a simple method for calculating similarity. You are given a dataset where each document is an excerpt from a classic English-language book. Your task will consist of the following steps:

1. Cleaning the documents
2. Converting the documents into "feature vectors" in a data model
3. Comparing different documents by measuring the similarity between feature vectors

With that as background, let's go!

In [1]:
import os
import math

In [2]:
import re

# Part 0. Data cleaning

Recall that the dataset is a collection of book excerpts. Run the next three cells below to see what the raw data look like.

In [3]:
from problem_utils import read_files

books, excerpts = read_files("resource/asnlib/publicdata/data/")
print(f"{len(books)} books found: {books}")

10 books found: ['prideandprejudice', 'gatsby', '1984', 'littlewomen', 'olivertwist', 'janeeyre', 'hamlet', 'kiterunner', 'littleprince', 'prisonerofazkaban']


Here's an excerpt from one of the books, namely, [George Orwell's classic novel 1984](https://en.wikipedia.org/wiki/Nineteen_Eighty-Four).

In [4]:
print(f"{len(excerpts)} excerpts (type: {type(excerpts)})")

excerpt_1984 = excerpts[books.index('1984')]
print(f"\n=== Excerpt from the book, '1984' ===\n{excerpt_1984}")

10 excerpts (type: <class 'list'>)

=== Excerpt from the book, '1984' ===
It was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him. 
The hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The flat was seven flights up, and Winston, who was thirty-ni

## Normalizing Text

As with any text analysis problem we will need to clean up this data. Start by cleaning the text as follows:

* Convert all the letters to lowercase
* Retain only alphabetic and space-like characters in the text.

For example, the sentence,
```python 
    '''How many more minutes till I get to 22nd and D'or street?'''
``` 

becomes,
```python
    '''how many more minutes till i get to nd and dor street'''
```

**Exercise 0.a** (1 point). Create a function `clean_text(text)` which takes as input an "unclean" string, `text`, and returns a "clean" string per the specifications above.

In [5]:
def clean_text(text):
    assert (isinstance(text, str)), "clean_text expects a string as input"
    # Convert every letter to lowercase
    text = text.lower()
    # Regex that substitutes anything that is NOT alphabetic or a space with a blank
    # Could also replace with text.lower() and skip the first step
    final = re.sub(r'[^a-zA-Z\s]', '', text)
    print(final)
    return final

In [6]:
# Test cell: `test_clean_text` (0.5 point)

# A few test cases:
print("Running fixed tests...")
sen1 = "How many more minutes till I get to 22nd and, D'or street?"
ans1 = "how many more minutes till i get to nd and dor street"

assert (isinstance(clean_text(sen1), str)), "Incorrect type of output. clean_text should return string."
assert (clean_text(sen1) == ans1), "Text incorrectly normalised. Output looks like '{}'".format(clean_text(sen1))

sen2 = "This is\n a whitespace\t\t test with 8 words."
ans2 = "this is\n a whitespace\t\t test with  words"
assert (clean_text(sen2) == ans2), "Text incorrectly normalised. Output looks like '{}'".format(clean_text(sen2))
print("==> So far, so good.")

# Some random instances;
def check_clean_text_random(max_runs=100):
    from random import randrange, random, choice
    def rand_run(options, max_run=5, min_run=1):
        return ''.join([choice(options) for _ in range(randrange(min_run, max_run))])
    printable = [chr(k) for k in range(33, 128) if k is not ord('\r')]
    alpha_lower = [c for c in printable if c.isalpha() and c.islower()]
    alpha_upper = [c.upper() for c in alpha_lower]
    non_alpha = [c for c in printable if not c.isalpha()]
    spaces = [' ', '\t', '\n']
    s_in = ''
    s_ans = ''
    for _ in range(randrange(0, max_runs)):
        p = random()
        if p <= 0.5:
            fragment = rand_run(alpha_lower)
            fragment_ans = fragment
        elif p <= 0.75:
            fragment = rand_run(alpha_upper)
            fragment_ans = fragment.lower()
        elif p <= 0.9:
            fragment = rand_run(non_alpha)
            fragment_ans = ''
        else:
            fragment = rand_run(spaces, max_run=3)
            fragment_ans = fragment
        s_in += fragment
        s_ans += fragment_ans
    print(f"\n* Input: {s_in}")
    s_you = clean_text(s_in)
    assert s_you == s_ans, f"ERROR: Your output is incorrect: '{s_you}'."

print("\nRunning battery of random tests...")
for _ in range(20):
    check_clean_text_random()

print("\n(Passed!)")

Running fixed tests...
how many more minutes till i get to nd and dor street
how many more minutes till i get to nd and dor street
this is
 a whitespace		 test with  words
==> So far, so good.

Running battery of random tests...

* Input: 	iddqxdvwtvov3i	BQPKEujlcRiux		dvggyz.1lGEBNv%rcfuhonxhfsrsHNOECgwcpvtvAbrkoxTT8-)iNDRHOYWUMXLuib		wmkjFMV;RNQTEFbbkQXITEUwuua
	iddqxdvwtvovi	bqpkeujlcriux		dvggyzlgebnvrcfuhonxhfsrshnoecgwcpvtvabrkoxttindrhoywumxluib		wmkjfmvrnqtefbbkqxiteuwuua

* Input: ombzzskyte
'<tnwtADVUpit	 tktybJWFim
 wlmdRLUCapqypnauwbDTQDbnqRUWEe?-_|rtpui	cqzcfb'"32pc$<1?Tgeg/`+$MTlxks]-:azes_%:*.]` 	rxswgvwkpxlm'81divNCHZZrtbpyyuBWYBMScGTCO7_?pescGPZC
xuvoohachflbrUJJC	qgvev	
aoupsCYUZAHyqnh
ombzzskyte
tnwtadvupit	 tktybjwfim
 wlmdrlucapqypnauwbdtqdbnqruweertpui	cqzcfbpctgegmtlxksazes 	rxswgvwkpxlmdivnchzzrtbpyyubwybmscgtcopescgpzc
xuvoohachflbrujjc	qgvev	
aoupscyuzahyqnh

* Input: :<0SZYYgkwf5%NUPZtjfwqkgygyqxk{(^pd27?$evaad'8 gvHYBYgDFRgfzhvaz	
?!7BXBLfnc
BVHFdcwvalhbyq


Let's clean some excerpts!  

**Exercise 0.b** (1 point). Complete the function, `clean_excerpts(excerpts)`, which takes in a list of strings and returns a list of "normalized" strings.

> Note: `clean_excerpts` should return a list of strings.

In [7]:
def clean_excerpts(excerpts):
    assert isinstance(excerpts, list), "clean_excerpts expects a list of strings as input"
    # Applies the above logic to a LIST of strings.
    # Could also use [clean_text() for s in excerpts]
    cleaned =  [re.sub(r'[^\sa-zA-Z]', '', item).lower().strip() for item in excerpts]
    print(type(cleaned))
    return cleaned


Run the following cells to clean our collection of excerpts.

In [8]:
docs = clean_excerpts(excerpts)

<class 'list'>


In [9]:
# Test Cell: `test_clean_excerpts` (1 point)

docs = clean_excerpts(excerpts)

puncts = ['‘', '…', '’', '—', ',', '”', '1', '“', '9', '5', '=', '?', '3', '!', ';', '"', '(', '-', ':', ')', '_', '0', '7', '.', "'"]
assert (isinstance(docs, list)), "Incorrect type of output. clean_excerpts should return a list of strings."
assert (len(docs) == len(excerpts)), "Incorrect number of cleaned excerpts returned."

for doc in docs:
    for c in doc:
        if c in puncts:
            assert False, "{} found in cleaned documents".format(c)
            
print("\n(Passed!)")

<class 'list'>

(Passed!)


# Part 1. Bag-of-Words

To calculate similarity between two documents, a well-known technique is the _bag-of-words_ model. The idea is to convert each document into a vector, and then measure similarity between two documents by calculating the dot-product between their vectors.

Here is how the procedure works. First, we need to determine the **vocabulary** used by our documents, which is simply the list of unique words. For instance, suppose we have the following two documents:
* `doc1 = "create ten different different sample"`
* `doc2 = "create ten another another example example example"`

Then the vocabulary is

* `['another', 'create', 'different', 'example', 'sample', 'ten']`

Next, let's create a **feature vector** for each document. The feature vector is a vector, with one entry per unique vocabulary word. The value of each entry is the number of times the word occurs in the document. For example, the feature vectors for our two sample documents would be:

```python 
vocabulary = ['another', 'create', 'different', 'example', 'sample', 'ten']
doc1_features =  [0, 1, 2, 0, 1, 1]
doc2_features = [2, 1, 0, 3, 0, 1]
```

> _Aside_: For a deeper dive into the bag-of-words model, refer to this [Wikipedia article](https://en.wikipedia.org/wiki/Bag-of-words_model). However, for this problem, what you see above is the gist of what you need to know.

### Stop Words

Not all words carry useful information for the purpose of a given analysis. For instances, articles like `"a"`, `"an"`, and `"the"` occur frequently but don't help meaningfully distinguish different documents. Therefore, we might want to omit them from our vocabulary.

Suppose we have decided that we have determined the list, `stop_words`, defined below, to be such a Python set of stop words.


In [10]:
stop_words = {'a', 'able', 'about', 'across', 'after', 'all', 'almost', 'also', 'am', 'among', 'an', 'and', 'any',
              'are', 'as', 'at', 'be', 'because', 'been', 'but', 'by', 'can', 'cannot', 'could', 'dear', 'did',
              'do', 'does', 'either', 'else', 'ever', 'every', 'for', 'from', 'get', 'got', 'had', 'has', 'have',
              'he', 'her', 'hers', 'him', 'his', 'how', 'however', 'i', 'if', 'in', 'into', 'is', 'it', 'its',
              'just', 'least', 'let', 'like', 'likely', 'may', 'me', 'might', 'most', 'must', 'my', 'neither',
              'no', 'nor', 'not', 'of', 'off', 'often', 'on', 'only', 'or', 'other', 'our', 'own', 'rather',
              'said', 'say', 'says', 'she', 'should', 'since', 'so', 'some', 'than', 'that', 'the', 'their',
              'them', 'then', 'there', 'these', 'they', 'this', 'tis', 'to', 'too', 'twas', 'us', 'wants',
              'was', 'we', 'were', 'what', 'when', 'where', 'which', 'while', 'who', 'whom', 'why', 'will',
              'with', 'would', 'yet', 'you', 'your'}

**Excercise 1.a** (1 point) Complete the function `extract_words(doc)`, below. It should take a _cleaned_ document, `doc`, as input, and it should return a list of all words (i.e., it should return a list of strings) subject to the following two conditions:

1. It should omit any stop words, i.e., it should return only "informative" words.
2. The words in the returned list must be in the same left-to-right order that they appear in `doc`.
3. The function must return all words, even if they are duplicates.

For instance:

```python
    # Omit stop words!
    extract_words("what is going to happen to me") == ['going', 'happen']
    
    # Return all words in-order, preserving duplicates:
    extract_words("create ten another another example example example") \
        == ['create', 'ten', 'another', 'another', 'example', 'example', 'example']
```

In [11]:
def extract_words(doc):
    assert isinstance(doc, str), "extract_words expects a string as input"
    # Returns all the words in our document split by spaces if not in the stop words set
    return [word for word in doc.split() if word not in stop_words]

In [12]:
# Test Cell: `test_extract_words` (1 point)

doc1 = "create ten different different sample"
doc2 = "create ten another another example example example"
doc_list = [doc1, doc2]

sen1 = doc1
ans1 = ['create', 'ten', 'different', 'different', 'sample']
assert(isinstance(extract_words(sen1),list)), "Incorrect type of output. extract_words should return a list of strings."
assert(extract_words(sen1) == ans1), "extract_words failed on {}".format(sen1)

sen2 = "what is going to happen to me"
ans2 = ['going', 'happen']
assert(extract_words(sen2) == ans2), "extract_words failed on {}".format(sen2)

print("\n (Passed!)")


 (Passed!)




**Exercise 1.b** (1 point). Next, let's create a vocabulary for the book-excerpt dataset.

Complete the function, `create_vocab(list_of_documents)`, below. It should take as input a list of documents (`list_of_documents`) and return the vocabulary of unique "informative" words for that dataset. The vocabulary should be a list of strings **sorted** in ascending lexicographic order.

For instance:

```python
doc1 = "create ten different different sample"
doc2 = "create ten another another example example example"
doc_list = [doc1, doc2]
create_vocab(doc_list) == ['another', 'create', 'different', 'example', 'sample', 'ten']
```

> **Note 0.** We do not want any stop words in the vocabulary. Make use of `extract_words()`!

In [13]:
def create_vocab(list_of_documents):
    assert isinstance(list_of_documents, list), "create_vocab expects a list as input."
    # Get every word for every document in the list and then iterate over every word from the list we get from extracting the document
    v_set = {word for doc in list_of_documents for word in extract_words(doc)}
    # We need to convert the set into a list and then sort it
    return sorted(list(v_set))


In [14]:
# Test Cell: `test_create_vocab` (1 point)

doc1 = doc_list
ans1 = ['another', 'create', 'different', 'example', 'sample', 'ten']
assert(isinstance(create_vocab(doc1),list)), "Incorrect type of output. create_vocab should return a list of strings."
assert(create_vocab(doc1) == ans1), "create_vocab failed on {}".format(doc1)

doc2 = [docs[books.index('gatsby')]]
ans2 = ['abnormal', 'abortive', 'accused', 'admission', 'advantages', 'advice', 'afraid', 'again', 'always', 'anyone', 'appears', 'attach', 'attention', 'autumn', 'away', 'back', 'being', 'birth', 'boasting', 'book', 'bores', 'came', 'care', 'certain', 'closed', 'college', 'come', 'communicative', 'conduct', 'confidences', 'consequence', 'creative', 'criticizing', 'curious', 'deal', 'decencies', 'detect', 'didnt', 'dignified', 'dont', 'dreams', 'dust', 'earthquakes', 'east', 'elations', 'end', 'everything', 'excursions', 'exempt', 'express', 'extraordinary', 'father', 'feel', 'feigned', 'felt', 'few', 'find', 'flabby', 'floated', 'forever', 'forget', 'foul', 'found', 'founded', 'frequently', 'fundamental', 'gatsby', 'gave', 'gestures', 'gift', 'gives', 'glimpses', 'gorgeous', 'great', 'griefs', 'habit', 'hard', 'havent', 'heart', 'heightened', 'hope', 'horizon', 'hostile', 'human', 'im', 'impressionability', 'inclined', 'infinite', 'interest', 'intimate', 'intricate', 'itself', 'ive', 'judgements', 'last', 'levity', 'life', 'limit', 'little', 'machines', 'made', 'man', 'many', 'marred', 'marshes', 'matter', 'meant', 'men', 'miles', 'mind', 'missing', 'moral', 'more', 'name', 'natures', 'never', 'normal', 'nothing', 'obvious', 'one', 'opened', 'out', 'over', 'parceled', 'people', 'person', 'personality', 'plagiaristic', 'point', 'politician', 'preoccupation', 'preyed', 'privileged', 'privy', 'promises', 'quality', 'quick', 'quivering', 'reaction', 'readiness', 'realized', 'register', 'related', 'remember', 'repeat', 'represented', 'reserve', 'reserved', 'reserving', 'responsiveness', 'revelation', 'revelations', 'right', 'riotous', 'rock', 'romantic', 'scorn', 'secret', 'sense', 'sensitivity', 'series', 'shall', 'shortwinded', 'sign', 'sleep', 'snobbishly', 'something', 'sorrows', 'sort', 'still', 'successful', 'such', 'suggested', 'suppressions', 'temperament', 'temporarily', 'ten', 'terms', 'those', 'thousand', 'told', 'tolerance', 'turned', 'turning', 'unaffected', 'unbroken', 'under', 'understood', 'unequally', 'uniform', 'unjustly', 'unknown', 'unmistakable', 'unsought', 'unusually', 'up', 'usually', 'veteran', 'victim', 'vulnerable', 'wake', 'wanted', 'way', 'wet', 'weve', 'whenever', 'wild', 'world', 'years', 'young', 'younger', 'youve']
assert(create_vocab(doc2) == ans2), "create_vocab failed on {}".format(doc2)

print("\n (Passed!)")


 (Passed!)


**Exercise 1.c** (2 points). Given a list of documents and a vocabulary, let's create bag-of-words vectors for each document.

Complete the function `bagofwords(doclist, vocab)`, below. It takes as input a list of documents (`doclist`) and a list of vocabulary words (`vocab`). It will return a list of bag-of-words vectors, with one vector for each document in the input.

For instance:

```python
doc1 = "create ten different different sample"
doc2 = "create ten another another example example example"
doc_list = [doc1, doc2]
vocab = ['another', 'create', 'different', 'example', 'sample', 'ten']
bagofwords(doc_list, vocab) == [[0, 1, 2, 0, 1, 1],
                                [2, 1, 0, 3, 0, 1]]
```

> **Note 0**: Every word in the document must be present in the vocabulary. Therefore you should use the same preprocessing function (`extract_words()`) that was used to create the vocabulary.
>
> **Note 1**: `bagofwords()` should return a list of vectors, where each vector is a list of integers.

In [15]:
def bagofwords(doclist, vocab):
    assert (isinstance(doclist, list)), "bagofwords expects a list of strings as input for doclist."
    assert (isinstance(vocab, list)), "bagofwords expects a list of strings as input for vocab."
    
    # i = indexes
    # w = words
    # enumerate() assigns a number to each element to the list, which we are doing to our vocab list
    # This creates a dictionary of index, word pairs for each element in vocab
    word_map = {w:i for i, w in enumerate(vocab)}
    # Creates nested list
    # Inner list for every document, inner list is 0,
    # then multiply by the length so that there is one entry for every element
    # of the vocabulary
    bow = [[0]*len(vocab) for _ in range(len(doclist))] #empty bag of words to be iterated over
    # I = row that we're on (think of the inner list as row)
    # doc = (think of the outer list as the columns)
    for i, doc in enumerate(doclist):
        # Words extracted from the document
        for word in extract_words(doc): #previous function
            # Inner index is for document i that we're on
            # Outer index is for the word in that document we're on
            # Add one to each word in the document
            bow[i][word_map[word]] += 1
    return bow


In [16]:
# Test Cell: `test_bagofwords_1` (1 point)

doc1 = doc_list
vocab1 = create_vocab(doc1)
vec1 = [0, 1, 2, 0, 1, 1]
assert(isinstance(bagofwords(doc1, vocab1),list)), "Incorrect type of output. bagofwords should return a list of integers."
assert(bagofwords(doc1, vocab1)[0] == vec1), "bagofwords failed on {}".format(doc1)

doc2 = [docs[books.index('1984')][-200:]]
vocab2 = create_vocab(doc2)
vec2 = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
assert(bagofwords(doc2, vocab2)[0] == vec2), "bagofwords failed on {}".format(doc2)

print("\n (Passed!)")


 (Passed!)


In [17]:
# Test Cell: `test_bagofwords_2` (1 point)

print("""
This test cell will be replaced with one hidden test case.
You will only know the result after submitting to the autograder.
If the autograder times out, then either your solution is highly
inefficient or contains a bug (e.g., an infinite loop).
""")

###
### AUTOGRADER TEST - DO NOT REMOVE
###



This test cell will be replaced with one hidden test case.
You will only know the result after submitting to the autograder.
If the autograder times out, then either your solution is highly
inefficient or contains a bug (e.g., an infinite loop).



Let us take a look at the number of words found in our BoW vectors.

In [18]:
for i in range(len(books)):
    bow = bagofwords(docs, create_vocab(docs))
    print('{:17s}\t: {} words'.format(books[i],len(bow[i])-bow[i].count(0)))

prideandprejudice	: 335 words
gatsby           	: 212 words
1984             	: 401 words
littlewomen      	: 212 words
olivertwist      	: 415 words
janeeyre         	: 126 words
hamlet           	: 519 words
kiterunner       	: 142 words
littleprince     	: 270 words
prisonerofazkaban	: 537 words


## Normalization (Again?)
One of the artifacts you might have noticed from the BoW vectors is that they have very different number of words. This is because the excerpts are of different lengths which may artificially skew the norms of these vectors.  

One way to remove this bias is to keep the direction of the vector but normalize the lengths to be equal to one. If the vector is $\mathbf{v} = \begin{bmatrix} v_0\\ v_1\\ \vdots\\ v_{n-1}\end{bmatrix} \in \mathbf{R}^n$, then its unit-normalized version is $\mathbf{\hat{v}}$, given by
  
$$
\mathbf{\hat{v}} = \frac{\mathbf{v}}{\lVert \mathbf{v}\rVert_2} = \frac{\mathbf{v}}{\sqrt{v_0^2 + v_1^2 + \ldots + v_{n-1}^2}}.
$$

For instance, recall the BoW vectors from our earlier example:
```python
bow = [[0, 1, 2, 0, 1, 1],
       [2, 1, 0, 3, 0, 1]]
```

The normalized versions would be
```python
bow_normalize = [[0.0, 0.3779644730092272, 0.7559289460184544, 0.0, 0.3779644730092272, 0.3779644730092272],
                 [0.5163977794943222, 0.2581988897471611, 0.0, 0.7745966692414834, 0.0, 0.2581988897471611]]
```

**Exercise 1.d** (2 points). Complete the function `bow_normalize(bow)`, below. It should take as input a list of BoW vectors. It should return their unit-normalized versions, per the formula above, also as a **list of vectors**.

In [19]:
def bow_normalize(bow):
    assert(isinstance(bow,list)),"bow_normalize expects a list of ints as input"
    
    # Looking to divide the value by the norm
    def norm(x):
        # Squared value for each value in the object divided by 1/2
        return sum([xi**2 for xi in x]) ** (1/2)
    # Apply the above function to every word in the bag of words
    bag_norm = [norm(b) for b in bow]
    # Nested list again. I = index, bag = element to access the norm
    # The inner list divides the value by the normalized value for each element
    return [[w/bag_norm[i] for w in bag] for i, bag in enumerate(bow)]
    


In [20]:
bow0 = [[1, 2, 3, 1, 1], [2, 2, 2, 2, 0]]
nbow0 = [[0.25, 0.5, 0.75, 0.25, 0.25], [0.5, 0.5, 0.5, 0.5, 0]]

ans0 = bow_normalize(bow0)
assert(isinstance(ans0,list)), "Incorrect type of output. bow_normalize should return a list of floats."

assert(nbow0[0] == ans0[0]), "bow_normalize failed on {}".format(bow0[0])
assert(nbow0[1] == ans0[1]), "bow_normalize failed on {}".format(bow0[1])

bow1 = [[0, 1, 2, 0, 1, 1],
       [2, 1, 0, 3, 0, 1]]
nbow1 = [[0.0, 0.3779644730092272, 0.7559289460184544, 0.0, 0.3779644730092272, 0.3779644730092272],
    [0.5163977794943222, 0.2581988897471611, 0.0, 0.7745966692414834, 0.0, 0.2581988897471611]]

ans1 = bow_normalize(bow1)
assert(nbow1[0] == ans1[0]), "bow_normalize failed on {}".format(bow1[0])
assert(nbow1[1] == ans1[1]), "bow_normalize failed on {}".format(bow1[1])

print("==> So far, so good.")
# Some Random Instances
def check_bow_normalize_random():
    from random import choice, sample
    
    vec_len =  choice(range(2,8))
    nvecs = choice(range(3,7))
    rvecs = []
    for _ in range(nvecs):
        rvecs.append(sample(range(2*vec_len),vec_len))
        
    unit_rvecs = bow_normalize(rvecs)
    
    for i in range(len(unit_rvecs)):
        print("Input {}".format(rvecs[i]))
        u = unit_rvecs[i]
        ans = 1.0;
        
        for ui in u:
            ans = ans - (ui*ui)
            
        assert (math.isclose(ans,0,rel_tol=1e-6,abs_tol=1e-8)), "ERROR: Your output is incorrect {}".format(u)
        
print("\nRunning battery of random tests...")
for _ in range(20):
    check_bow_normalize_random()

print("\n (Passed!)")

==> So far, so good.

Running battery of random tests...
Input [2, 12, 0, 4, 9, 13, 6]
Input [8, 1, 2, 5, 4, 3, 7]
Input [6, 12, 13, 4, 2, 11, 10]
Input [3, 10, 5, 7, 6, 12, 0]
Input [6, 0, 7, 2, 1]
Input [1, 9, 8, 4, 0]
Input [2, 6, 4, 1, 5]
Input [5, 9, 8, 7, 1]
Input [1, 6, 7, 4, 8]
Input [7, 3, 4, 1, 5]
Input [6, 5, 0, 4]
Input [1, 5, 4, 7]
Input [1, 6, 0, 2]
Input [3, 7, 2, 1]
Input [0, 2, 7, 4]
Input [1, 11, 0, 8, 4, 5]
Input [1, 7, 4, 10, 3, 8]
Input [4, 0, 5, 10, 7, 9]
Input [0, 6, 3, 7, 10, 1]
Input [1, 7, 11, 10, 2, 8]
Input [3, 1, 6, 5, 13, 7, 9]
Input [12, 9, 4, 1, 5, 2, 3]
Input [5, 6, 4, 12, 9, 8, 1]
Input [8, 0, 2, 11, 10, 1, 4]
Input [9, 11, 7, 10, 4, 0, 1]
Input [1, 2, 4, 0, 11, 12, 5]
Input [1, 2, 4]
Input [3, 0, 4]
Input [4, 0, 5]
Input [3, 4, 5]
Input [0, 5, 4, 1, 9]
Input [1, 8, 9, 0, 6]
Input [8, 4, 3, 7, 5]
Input [3, 8, 0, 1, 4]
Input [1, 3, 7, 0]
Input [5, 1, 7, 2]
Input [6, 2, 1, 7]
Input [0, 6, 4, 5]
Input [3, 1, 0, 5]
Input [4, 5, 2, 0]
Input [4, 5, 0, 3]
Inp

(_Aside_) **Sparsity of BoW vectors.** As an aside, run the next cell to see the BoW vectors are actually quite _sparse_.

In [21]:
literature_vocab = create_vocab(docs)
bow = bagofwords(docs, literature_vocab)
nbow = bow_normalize(bow)
numterms = len(docs)*len(literature_vocab)
numzeros = sum([b.count(0) for b in nbow])
print("Percentage of entries which are zero: {:.1f} %".format(100*numzeros/numterms))

Percentage of entries which are zero: 85.9 %


> If everything is correct, you'll see that the BoW vectors are sparse, with about 85-86% of the components being zeroes. Therefore, we could in principle save a lot of space by only storing the non-zeroes. While we do not exploit this fact in our current example it is useful to think about these costs while running analytics at scale.

# Part 2. Comparing Documents

Now we have normalized vector versions of each document, we can use a standard similarity measure to compare vectors. For this question, we shall use the *inner product*. Recall that the inner product of two vectors $a,b \in \mathbf{R}^n$ is defined as,

$$<a,b> = \Sigma_{i=0}^{n-1} a_i b_i$$

For example,

$$<[1,-1,3], [2,4,-1]> = (1\times2)+(-1\times4)+(3\times-1)=-5$$

**Exercise 2.a** (1 point) Complete the function `inner_product(a, b)` which takes two vectors, `a` and `b`, both represented as lists, and returns their inner product.

> Note: `inner_product(a, b)` should return a value of type `float`.

In [22]:
def inner_product(a,b):
    assert (isinstance(a, list)), "inner_product expects a list of floats/ints as input for a."
    assert (isinstance(b, list)), "inner_product expects a list of floats/ints as input for b."
    assert len(a) == len(b), "inner_product should be called on vectors of the same length."
    
    # Pair of corresponding elements of each vector, take the product
    # Result has to be a float
    return sum([float(ai * bi) for ai, bi in zip(a, b)])


In [23]:
# Test Cell: `test_inner_product` (0.5 point)

vec1a = [1,-1,3]
vec1b = [2,4,-1]
ans1 = -5
assert (isinstance(inner_product(vec1a,vec1b),float)), "Incorrect type of output. inner_product should return a float."
assert (inner_product(vec1a,vec1b) == ans1), "inner_product failed on inputs {} and {}".format(vec1a,vec1b)
assert (inner_product(vec1b,vec1a) == ans1), "inner_product failed on inputs {} and {}".format(vec1b,vec1a)

vec2a = [0,2,1,9,-1]
vec2b = [17,4,1,-1,0]
ans2 = 0
assert (inner_product(vec2a,vec2b) == ans2), "inner_product failed on inputs {} and {}".format(vec2a,vec2b)
assert (inner_product(vec2b,vec2a) == ans2), "inner_product failed on inputs {} and {}".format(vec2b,vec2a)

print("\n (Passed!)")


 (Passed!)


We can use the `inner_product()` as a measure of similarity between documents! (_Recall the linear algebra refresher in Topic 3_.) In particular, since our normalized BoW vectors are "direction" vectors, the inner product measures how closely two vectors point in the same direction.

**Exercise 2.b** (1 point). Now we can finally answer our initial question: which book excerpts are similar to each other? Complete the function `most_similar(nbows, target)`, below, to answer this question. In particular, it should take as input the normalized BoW vectors created in the previous part, as well as a target excerpt index $i$. It should return most index of the excerpt most similar to $i$.

> **Note 0.** Ties in scores are won by the smaller index. For example, if excerpt 2 and excerpt 7 both equally similar to the target excerpt 8, then return 2 as the most similar excerpt.
>
> **Note 1.** Your `most_similar()` function should return a value of type `int`.

> **Note 2.** The test cell refers to hidden tests, but in fact, the test is not hidden per se. Instead, we are hashing the strings returned by your solution to be able to check your answer without revealing it to you directly.

In [24]:
def most_similar(nbows, target):
    assert (isinstance(nbows,list)), "most_similar expects list as input for nbows."
    assert (isinstance(target,int)), "most_similar expects integer as input for target."
    
    sim_tuples = [(i, inner_product(bow, nbows[target])) \
                  # For every index, bag in enumerated nbows element
                  for i, bow in enumerate(nbows)  
                  if i != target]
    return max(sim_tuples, key = lambda x: x[1])[0]


In [25]:
# Test Cell: `test_most_similar` (1 point)
literature_vocab = create_vocab(docs)
bow = bagofwords(docs, literature_vocab)
nbow = bow_normalize(bow)

# Start with two basic cases:
target1 = books.index('1984') 
ans1 = books.index('kiterunner')
assert (isinstance(most_similar(nbow,target1),int)), "most_similar should return integer."
assert (most_similar(nbow,target1) == ans1), "most_similar failed on input {}".format(books[target1])

target2 = books.index('prideandprejudice')
ans2 = books.index('hamlet')
assert (most_similar(nbow,target2) == ans2), "most_similar failed on input {}".format(books[target2])

# Check the rest via obscured, hashed solutions
###
### AUTOGRADER TEST - DO NOT REMOVE
###
def check_most_similar_solns():
    from problem_utils import make_hash, open_file
    literature_vocab = create_vocab(docs)
    bow = bagofwords(docs, literature_vocab)
    nbow = bow_normalize(bow)
    with open_file("most_similar_solns.csv", "rt") as fp_soln:
        for line in fp_soln.readlines():
            target_name, soln_hashed = line.strip().split(',')
            target_id = books.index(target_name)
            your_most_sim_id = most_similar(nbow, target_id)
            assert isinstance(your_most_sim_id, int), f"Your function returns a value of type {type(your_most_sim_id)}, not an integer"
            assert 0 <= your_most_sim_id < len(nbow), f"You returned {your_most_sim_id}, which is an invalid value (it should be between 0 and {nbow})"
            your_most_sim_name = books[your_most_sim_id]
            print(f"For book '{target_name}', you calculated '{your_most_sim_name}' as most similar.")
            your_most_sim_name_hashed = make_hash(your_most_sim_name)
            assert your_most_sim_name_hashed == soln_hashed, "==> ERROR: Unfortunately, your returned value does not appear to match our reference solution."

check_most_similar_solns()
print("\n (Passed!)")

For book '1984', you calculated 'kiterunner' as most similar.
For book 'gatsby', you calculated 'prideandprejudice' as most similar.
For book 'hamlet', you calculated 'prideandprejudice' as most similar.
For book 'janeeyre', you calculated 'prideandprejudice' as most similar.
For book 'kiterunner', you calculated '1984' as most similar.
For book 'littleprince', you calculated 'littlewomen' as most similar.
For book 'littlewomen', you calculated 'littleprince' as most similar.
For book 'olivertwist', you calculated '1984' as most similar.
For book 'prideandprejudice', you calculated 'hamlet' as most similar.
For book 'prisonerofazkaban', you calculated '1984' as most similar.

 (Passed!)


Now let's have a look at the documents most similar to each other, according to your implementation.

In [26]:
for idx in range(len(books)):
    jdx = most_similar(nbow,idx)
    print(books[idx],"is most similar to",books[jdx],"!")

prideandprejudice is most similar to hamlet !
gatsby is most similar to prideandprejudice !
1984 is most similar to kiterunner !
littlewomen is most similar to littleprince !
olivertwist is most similar to 1984 !
janeeyre is most similar to prideandprejudice !
hamlet is most similar to prideandprejudice !
kiterunner is most similar to 1984 !
littleprince is most similar to littlewomen !
prisonerofazkaban is most similar to 1984 !


**Fin!** You’ve reached the end of this part. Don’t forget to restart and run all cells again to make sure it’s all working when run in sequence; and make sure your work passes the submission process. Good luck!